In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

file_path = 'C:/Users/marya/OneDrive/Desktop/Classification_project1/preprocessed_green_tripdata_2015-07.csv'
data = pd.read_csv(file_path)

# Create the price_category column based on total_amount quantiles
quantiles = data['total_amount'].quantile([0.33, 0.66])
def categorize_price(amount):
    if amount <= quantiles[0.33]:
        return 'Low'
    elif amount <= quantiles[0.66]:
        return 'Medium'
    else:
        return 'High'

data['price_category'] = data['total_amount'].apply(categorize_price)

y = data['price_category']

# Drop or convert non-numeric columns
non_numeric_columns = ['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime']
data = data.drop(columns=non_numeric_columns)

# Convert categorical variables to numeric using one-hot encoding
X = pd.get_dummies(data.drop('price_category', axis=1), drop_first=True)

# Check for missing values and impute them
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [2]:
# Perform k-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_val_score(rf_classifier, X, y, cv=kf, scoring='accuracy')

accuracies = []
precisions = []
recalls = []
conf_matrices = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    rf_classifier.fit(X_train, y_train)
    y_pred = rf_classifier.predict(X_test)
    
    accuracies.append(accuracy_score(y_test, y_pred))
    precisions.append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
    recalls.append(recall_score(y_test, y_pred, average='weighted'))
    conf_matrices.append(confusion_matrix(y_test, y_pred))


In [3]:
avg_accuracy = sum(accuracies) / len(accuracies)
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_conf_matrix = sum(conf_matrices)

class_report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(class_report).transpose()

In [4]:
print("Class  Accuracy  Precision    Recall")
for i, category in enumerate(['Low', 'Medium', 'High']):
    print(f"{i}     {category}  {report_df.loc[category, 'precision']:.6f}  {report_df.loc[category, 'recall']:.6f}")

print(f"Average Testing Accuracy: {avg_accuracy}")
print(f"Average Testing Precision: {avg_precision}")
print(f"Average Testing Recall: {avg_recall}")
print("Average Confusion Matrix:")
print(avg_conf_matrix)

print(f"True Low:  {conf_matrices[-1][0, 0]} False Medium:  {conf_matrices[-1][0, 1]} False High:  {conf_matrices[-1][0, 2]}")
print(f"False Low:  {conf_matrices[-1][1, 0]} True Medium:  {conf_matrices[-1][1, 1]} False High:  {conf_matrices[-1][1, 2]}")
print(f"False Low:  {conf_matrices[-1][2, 0]} False Medium:  {conf_matrices[-1][2, 1]} True High:  {conf_matrices[-1][2, 2]}")

Class  Accuracy  Precision    Recall
0     Low  1.000000  1.000000
1     Medium  0.999897  1.000000
2     High  1.000000  0.999902
Average Testing Accuracy: 0.9999545778989034
Average Testing Precision: 0.9999545831857505
Average Testing Recall: 0.9999545778989034
Average Confusion Matrix:
[[101799      0     11]
 [     0 108874      1]
 [     0      2  97533]]
True Low:  20360 False Medium:  0 False High:  2
False Low:  0 True Medium:  21775 False High:  0
False Low:  0 False Medium:  0 True High:  19507
